# Lab 6: Transformers and LLMs
```
- [S25] Advanced Machine Learning, Innopolis University
- Teaching Assistant: Gcinizwe Dlamini
```
<hr>


```
Lab Plan
1. Transformers (translation architecture)
2. Self-Attention
3. Multi-headed attention
4. Positional Encoding
5. Transfomer Encoder Part
6. Application of Transformers
7. Self practice tasks
```

<hr>


# 1. Transformers

* [Attention Is All You Need](https://arxiv.org/pdf/1706.03762.pdf) -- Original paper on attention

![](http://jalammar.github.io/images/t/The_transformer_encoder_decoder_stack.png)

## 1.1 Transfomer Encoder

The encoder contains a self-attention layer – a layer that helps the encoder look at other words in the input sentence as it encodes a specific word. <br>
PyTorch implementation : `nn.TransformerEncoder` and `nn.TransformerEncoderLayer` <br>
**The main goal is to efficiently encode the data**

  1         |  2
:-------------------------:|:-------------------------:
![](http://jalammar.github.io/images/t/encoder_with_tensors.png)  |  ![](http://jalammar.github.io/images/t/transformer_resideual_layer_norm_2.png)


## 2. Self-Attention

**Keep in mind : The main goal is to encode the data in a much more efficient way** In other words is to create meaningful embeddings<br>
- As the model processes each word (each position in the input sequence), self attention allows it to look at other positions in the input sequence for clues that can help lead to a better encoding for this word.


**How does Self-Attention work?**

Steps:
1. For each word, we create a **`Query`** vector, a **`Key`** vector, and a **`Value`** vector.
  - What are the **`Query`** vector, a **`Key`** vector, and a **`Value`** vector? : They're abstractions that are useful for calculating attention... They are a breakdown of the word embeddings
2. Calculating self-attention score from **`Query`** **`Key`** vector.
3. Divide the scores by 8 (This leads to having more stable gradients)
4. Pass the result through a softmax operation (softmax score determines how much each word will be expressed at this position)
5. Multiply each value vector by the softmax score
6. Sum up the weighted value vectors

### Step 1

For each word, we create a **`Query`** vector, a **`Key`** vector, and a **`Value`** vector.

![](http://jalammar.github.io/images/t/transformer_self_attention_vectors.png)

In [1]:
import torch

# simple sequence = I will pass AML midterm
simple_sequence_embedding = torch.rand((5, 512))

# Create weight matrices
Wq = torch.normal(0,0.5, (512, 7))
Wk = torch.normal(0,0.1, (512, 7))
Wv = torch.normal(0,0.2, (512, 7))


# Create key, query and value for each word in the senetence
queries = simple_sequence_embedding @ Wq
keys = simple_sequence_embedding @ Wk
values = simple_sequence_embedding @ Wv

### Step 2

Calculating self-attention score from **`Query`** and **`Key`** vector

In [2]:
scores = queries @ keys.T
scores

tensor([[30.1358, 18.4392, 29.8348, 40.2687, 42.2601],
        [17.9468,  2.4468, 16.2118, 18.4139, 19.1587],
        [26.3514, 15.4484, 33.9176, 44.3448, 40.3456],
        [11.0957, -6.5896,  3.6480, 16.1486,  6.7803],
        [29.3687, 25.3186, 33.1763, 60.3646, 55.5537]])

### Step 3
Divide the scores by 8 (This leads to having more stable gradients)

In [3]:
scores = scores / (keys.shape[-1] ** 0.5)
scores

tensor([[11.3903,  6.9694, 11.2765, 15.2201, 15.9728],
        [ 6.7833,  0.9248,  6.1275,  6.9598,  7.2413],
        [ 9.9599,  5.8390, 12.8196, 16.7608, 15.2492],
        [ 4.1938, -2.4906,  1.3788,  6.1036,  2.5627],
        [11.1003,  9.5695, 12.5394, 22.8157, 20.9973]])

### Step 4

Pass the result through a softmax operation

In [4]:
import torch.nn.functional as f
scores = f.softmax(scores, dim=-1)
scores

tensor([[6.8623e-03, 8.2508e-05, 6.1245e-03, 3.1606e-01, 6.7087e-01],
        [2.3278e-01, 6.6473e-04, 1.2082e-01, 2.7772e-01, 3.6801e-01],
        [8.9660e-04, 1.4551e-05, 1.5652e-02, 8.0573e-01, 1.7771e-01],
        [1.2487e-01, 1.5611e-04, 7.4802e-03, 8.4306e-01, 2.4438e-02],
        [7.0269e-06, 1.5204e-06, 2.9632e-05, 8.6033e-01, 1.3963e-01]])

### Step 5 & 6

* Multiply each value vector by the softmax score
* Sum up the weighted value vectors



In [5]:
z = scores @ values
z

tensor([[-1.7536e+00, -2.5648e+00,  1.2177e+00, -4.7075e+00, -1.1307e+00,
          3.4390e+00, -1.0637e+00],
        [-1.5368e+00, -2.6449e+00,  1.3245e+00, -4.7680e+00, -1.5274e+00,
          3.0759e+00, -1.0441e+00],
        [-8.2911e-01, -1.9917e+00, -1.8432e-01, -5.0140e+00, -1.1249e-01,
          3.5235e+00, -6.4149e-01],
        [-6.5421e-01, -1.9699e+00, -1.6939e-01, -5.2413e+00, -2.3667e-01,
          3.4747e+00, -5.0592e-01],
        [-7.3888e-01, -1.9283e+00, -3.1250e-01, -5.0841e+00, -4.7249e-03,
          3.5655e+00, -5.7742e-01]])

## 3. Multi-headed attention

**GOAL**:
1. Expand the model's ability to focus on different positions
2. Provide the attention layer multiple “representation subspaces”

**Attention with $N$ just means repeating self attention algorithm $N$ times and joining the results**


![](https://data-science-blog.com/wp-content/uploads/2022/01/mha_img_original.png)

**Multi-headed attention steps:**
1. Same as self-attention calculation, just n different times with different weight matrices
2. Condense the $N$ z metrices down into a single matrix by concatinating the matrices then multiply them by an additional weights matrix `WO`

Now the output z metrix is fed to the FFNN

In [6]:
from torch import Tensor
import torch.nn.functional as f
from torch import nn


def scaled_dot_product_attention(query, key, value):
    temp = query.bmm(key.transpose(1, 2))
    scale = query.size(-1) ** 0.5
    softmax = f.softmax(temp / scale, dim=-1)
    return softmax.bmm(value)

### 3.1 Attention head

In [7]:
class AttentionHead(nn.Module):
    def __init__(self, dim_in, dim_q, dim_k):
        super().__init__()
        self.q = nn.Linear(dim_in, dim_q)
        self.k = nn.Linear(dim_in, dim_k)
        self.v = nn.Linear(dim_in, dim_k)

    def forward(self, query, key, value):
        return scaled_dot_product_attention(self.q(query), self.k(key), self.v(value))

### 3.2 Multi Head Attention

**Task:** Implement multi-head attention model

In [8]:
class MultiHeadAttention(nn.Module):
  def __init__(self, number_of_heads, dim_in, dim_q, dim_k):
      super().__init__()
      self.heads = nn.ModuleList([AttentionHead(dim_in, dim_q, dim_k) for _ in range(number_of_heads)])
      self.linear = nn.Linear(number_of_heads * dim_k, dim_in)

  def forward(self, query, key, value):
      z = self.linear(torch.cat([h(query, key, value) for h in self.heads], dim=-1))
      return z


## 4. Positional Encoding

A way to account for the order of the words in the input sequence. A transformer adds a vector to each input embedding which helps it determine the position of each word. <br>
**Goal** : preserving information about the order of tokens  

Positional Encoding they can either be learned or fixed a priori.

Proposed approach from original paper : describe a simple scheme for fixed positional encodings based on sine and cosine functions

In [9]:
def position_encoding(seq_len, dim_model, device):
    pos = torch.arange(seq_len, dtype=torch.float, device=device).reshape(1, -1, 1)
    dim = torch.arange(dim_model, dtype=torch.float, device=device).reshape(1, 1, -1)
    phase = pos / (1e4 ** (dim / dim_model))

    return torch.where(dim.long() % 2 == 0, torch.sin(phase), torch.cos(phase))

![](http://jalammar.github.io/images/t/transformer_resideual_layer_norm_2.png)

## 5. Transfomer Encoder Part
### 5.1Encoder Feed Forward

In [10]:
def feed_forward(dim_input = 512, dim_feedforward = 2048):
    return nn.Sequential(nn.Linear(dim_input, dim_feedforward), nn.ReLU(), nn.Linear(dim_feedforward, dim_input))

### 5.2 Encoder Residual

From the original paper the author implementation

In [56]:
class Residual(nn.Module):
  def __init__(self, sublayer, dimension, dropout = 0.1):
    super().__init__()
    self.sublayer = sublayer
    self.norm = nn.LayerNorm(dimension)
    self.dropout = nn.Dropout(dropout)

  def forward(self, *tensors):
    return self.norm(tensors[0] + self.dropout(self.sublayer(*tensors)))

### 5.3 Putting the Encoder layer together

In [57]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, dim_model = 512, num_heads = 6, dim_feedforward = 2048, dropout = 0.1):
        super().__init__()
        dim_q = dim_k = max(dim_model // num_heads, 1)
        self.attention = Residual(MultiHeadAttention(num_heads, dim_model, dim_q, dim_k),
                                  dimension=dim_model, dropout=dropout)
        self.feed_forward = Residual(feed_forward(dim_model, dim_feedforward), dimension=dim_model, dropout=dropout)

    def forward(self, src):
        src = self.attention(src, src, src)
        return self.feed_forward(src)

## 5.4 Putting together transfomer Encoder part

In [58]:
class TransformerEncoder(nn.Module):
    def __init__(self, num_layers = 12, dim_model = 512, num_heads = 4, dim_feedforward = 2048, dropout: float = 0.1):
        super().__init__()
        self.layers = nn.ModuleList([TransformerEncoderLayer(dim_model, num_heads, dim_feedforward, dropout) for _ in range(num_layers) ])

    def forward(self, src):
        seq_len, dimension = src.size(1), src.size(2)
        src += position_encoding(seq_len, dimension, src.device)
        for layer in self.layers:
            src = layer(src)

        return src

# The Decoder Side

The encoder start by processing the input sequence. The output of the top encoder is then transformed into a set of attention vectors K and V. These are to be used by each decoder.

The “Encoder-Decoder Attention” layer works just like multiheaded self-attention, except it creates its Queries matrix from the layer below it, and takes the `Keys` and `Values` matrix from the output of the encoder stack.

![](https://jalammar.github.io/images/t/transformer_resideual_layer_norm_3.png)


## Decoder layer



**Task**: implement the decoder layer

In [59]:
class TransformerDecoderLayer(nn.Module):
    def __init__(self, dim_model=512, num_heads=6, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        dim_q = dim_k = max(dim_model // num_heads, 1)
        self.self_attention = Residual(MultiHeadAttention(num_heads, dim_model, dim_q, dim_k), dimension=dim_model, dropout=dropout)
        self.cross_attention = Residual(MultiHeadAttention(num_heads, dim_model, dim_q, dim_k), dimension=dim_model, dropout=dropout)
        self.feed_forward = Residual(feed_forward(dim_model, dim_feedforward), dimension=dim_model, dropout=dropout)

    def forward(self, tgt, memory, tgt_mask=None, memory_mask=None):
        tgt = self.self_attention(tgt, tgt, tgt, tgt_mask)
        tgt = self.cross_attention(tgt, memory, memory, memory_mask)
        return self.feed_forward(tgt)

## 6. Application of Transfomers

We will look at sentiment analysis

In [76]:
import transformers

class Transformer(nn.Module):
    def __init__(self, tag_count: int):
        super().__init__()
        self.transformer = transformers.AutoModel.from_pretrained('bert-base-uncased')
        for param in self.transformer.parameters():
            param.requires_grad = False
        hidden_dim = self.transformer.config.hidden_size
        self.fc = nn.Linear(hidden_dim, tag_count)

    def forward(self, input_ids: torch.Tensor) -> torch.Tensor:
        attention_mask = (input_ids != 0).long()
        output = self.transformer(input_ids, attention_mask=attention_mask, output_attentions=True)
        hidden = output.last_hidden_state  # [batch_size, seq_len, hidden_dim]
        logits = self.fc(torch.tanh(hidden))  # [batch_size, seq_len, tag_count]
        return logits

## 7. Tasks

```
Task 1
- Using the above implementation code the decoder layer and assemble a full transformer model
```

<hr>

```
Task 2
- Implement, train and test a Transfomer model (use pytorch layers)for Part-of-speech tagging task.
```

**Task 2 Datasets**: [Train](https://www.dropbox.com/s/x9n6f9o9jl7pno8/train_pos.txt?dl=1), [Test](https://www.dropbox.com/s/v8nccvq7jewcl8s/test_pos.txt?dl=1)


In [ ]:
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from sklearn.metrics import accuracy_score, classification_report

In [78]:
class POSTagDataset(Dataset):
    def __init__(self, filepath: str, max_len: int = 50, tag_vocab: dict = None):
        self.max_len = max_len
        self.sentences, self.tags = self.read_file(filepath)
        self.tokenizer = transformers.AutoTokenizer.from_pretrained('bert-base-uncased')
        if tag_vocab is None:
            self.tag_vocab = self.build_tag_vocab(self.tags)
        else:
            self.tag_vocab = tag_vocab
        self.encoded_data = self.encode_data()

    def read_file(self, filepath: str):
        sentences = []
        tags = []
        current_sentence = []
        current_tags = []
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if line == "":
                    if current_sentence:
                        sentences.append(current_sentence)
                        tags.append(current_tags)
                        current_sentence, current_tags = [], []
                else:
                    parts = line.split()
                    if len(parts) == 2:
                        word, tag = parts
                        current_sentence.append(word)
                        current_tags.append(tag)
        if current_sentence:
            sentences.append(current_sentence)
            tags.append(current_tags)
        return sentences, tags

    def build_tag_vocab(self, tag_lists) -> dict:
        counter = Counter()
        for tag_list in tag_lists:
            counter.update(tag_list)
        tag_vocab = {'<PAD>': 0}
        idx = 1
        for tag in sorted(counter.keys()):
            tag_vocab[tag] = idx
            idx += 1
        return tag_vocab

    def encode_data(self) -> list:
        encoded_data = []
        for sent, tag_seq in zip(self.sentences, self.tags):
            encoded = self.tokenizer(
                sent,
                is_split_into_words=True,
                max_length=self.max_len,
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )
            input_ids = encoded['input_ids'].squeeze(0)
            word_ids = encoded.word_ids()
            encoded_tags = []
            for word_idx in word_ids:
                if word_idx is None:
                    encoded_tags.append(self.tag_vocab['<PAD>'])
                else:
                    encoded_tags.append(self.tag_vocab[tag_seq[word_idx]])
            encoded_tags = torch.tensor(encoded_tags, dtype=torch.long)
            encoded_data.append({'input_ids': input_ids, 'tags': encoded_tags})
        return encoded_data

    def __len__(self) -> int:
        return len(self.encoded_data)

    def __getitem__(self, idx: int):
        example = self.encoded_data[idx]
        input_ids = example['input_ids']
        tags = example['tags']
        return input_ids, tags

In [ ]:
class POSTrainer:
    def __init__(self, model: nn.Module, learning_rate: float, tag_pad_idx: int):
        self.model = model
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.criterion = nn.CrossEntropyLoss(ignore_index=tag_pad_idx)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=learning_rate)
        print(f"Using device: {self.device}")

    def train(self, dataset: Dataset, num_epochs: int = 5, batch_size: int = 32):
        self.model.to(self.device)
        self.model.train()
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        for epoch in range(num_epochs):
            total_loss = 0.0
            for input_ids, tags in dataloader:
                input_ids, tags = input_ids.to(self.device), tags.to(self.device)
                self.optimizer.zero_grad()
                logits = self.model(input_ids) 
                loss = self.criterion(logits.view(-1, logits.shape[-1]), tags.view(-1))
                loss.backward()
                self.optimizer.step()
                total_loss += loss.item()
            print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader):.4f}')

    def evaluate(self, dataset: POSTagDataset, batch_size: int = 32):
        self.model.eval()
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
        all_preds = []
        all_tags = []
        with torch.no_grad():
            for input_ids, tags in dataloader:
                input_ids, tags = input_ids.to(self.device), tags.to(self.device)
                logits = self.model(input_ids)
                preds = logits.argmax(dim=-1)
                mask = tags != dataset.tag_vocab['<PAD>']
                all_preds.extend(preds[mask].cpu().numpy())
                all_tags.extend(tags[mask].cpu().numpy())
        accuracy = accuracy_score(all_tags, all_preds)
        print(f'POS Tagging Accuracy: {accuracy:.4f}')
        tag_mapping = {idx: tag for tag, idx in dataset.tag_vocab.items() if tag != '<PAD>'}
        labels = sorted(tag_mapping.keys())
        target_names = [tag_mapping[idx] for idx in labels]
        print(classification_report(all_tags, all_preds, labels=labels, target_names=target_names, zero_division=0))

In [80]:
train_filepath = "train_pos.txt"
test_filepath = "test_pos.txt"

pos_train_dataset = POSTagDataset(train_filepath, max_len=50)
pos_test_dataset = POSTagDataset(test_filepath, max_len=50, tag_vocab=pos_train_dataset.tag_vocab)

tag_count = len(pos_train_dataset.tag_vocab)
learning_rate = 0.001

model = Transformer(tag_count)
pos_trainer = POSTrainer(model, learning_rate, tag_pad_idx=pos_train_dataset.tag_vocab['<PAD>'])

pos_trainer.train(pos_train_dataset, num_epochs=5, batch_size=32)
pos_trainer.evaluate(pos_test_dataset, batch_size=32)

Using device: cuda
Epoch 1/5, Loss: 0.9787
Epoch 2/5, Loss: 0.4133
Epoch 3/5, Loss: 0.3432
Epoch 4/5, Loss: 0.3127
Epoch 5/5, Loss: 0.2952
POS Tagging Accuracy: 0.9239
              precision    recall  f1-score   support

           #       1.00      1.00      1.00         9
           $       1.00      1.00      1.00       378
          ''       1.00      1.00      1.00       575
           (       0.97      0.91      0.94       300
           )       0.93      0.95      0.94       281
           ,       0.97      1.00      0.99      2364
           .       0.93      0.99      0.96      1830
           :       0.99      0.99      0.99       342
          CC       0.99      0.99      0.99      1192
          CD       0.97      0.95      0.96      3195
          DT       0.97      0.98      0.98      3954
          EX       0.98      1.00      0.99        48
          FW       0.00      0.00      0.00         4
          IN       0.96      0.98      0.97      4973
          JJ       0.